# Thresholding, masking and preprocessing


Fluorescence datasets represent a relationship between the pixels in an image and the location and local density of your fluorescent molecule in a sample. However, properties of the detectors, optics, or even the samples can confound direct interpretation of this data. Here we will present some operations that can mitigate these effects to achieve robust hypothesis testing.

Today, we will explore ways to automatically define image masks -- for ROIs and other applications -- and deal with aberrations in the data that can make such analysis difficult on raw data

Episode 1: Load, understand and visualize the data  


Episode 2: Thresholding
> a) Global threshold and masking     
b) Automatic threshold detection  

Episode 3: Filtering and background subtraction
> a) The need for preprocessing  
b) Filtering out noise   
c) Background subtraction  

***
# Episode 1: Load, understand and visualize the data

First, let's import packages and set some plotting defaults


### Episode 1a) Load the images


Another helpful package for navigaving your filesystem is __os__  
https://docs.python.org/3/library/os.html#module-os

#### <font color='red'> Exercise 1</font>: load lesson 2 data  and check the shape of the data  
use imread from skimage.io to load the .tif data file

### Episode 1b) Load the metadata

Load the metadata, which is in JSON format

Take a look at the metadata dictionary

Let's re-label the slices with names, instead of numbers.

### Episode 1c) Visualize the images  
We'll use a slightly different plotting syntax to make __subplots__ to display each channel

***
# Episode 2: Thresholding and masking

Suppose we want to automatically select regions of interest (ROIs) from these images to, for example, count cells or measure the intensity of some fluorescent signal within the cells. One simple way to start to do this is by thresholding the image based on the pixel intensity.

To illustrate, we will focus on the YFP channel.   
Let's first split up our channels into descriptive variable names...

### Part 2a) Global thresholding and masking


Let's create a __mask__ for the channel 1 image   
How should we decide on the threshold value? First let's get a sense of the range of pixel intensities present in the image by looking at a histogram

OK now let's pick a threshold and make a mask...

Now we'll visualize the mask we computed

This is a start, but the idea was to *automate* this process. Fortunately there exist several auotmatic threshold calculation methods that are included in the scikit-image library. Generally these automatic threshold methods rely on the distribution of pixel intensity values.

### Part 2b) Automatic threshold detection  

Let's use some built-in automatic threshold algorithms to define a threshold for our channel 1 image  
https://scikit-image.org/docs/dev/api/skimage.filters.html  
https://en.wikipedia.org/wiki/Otsu%27s_method  


Visualize the image histogram and the automatically calculated threshold values...  

#### <font color='red'> Exercise 2</font>: make a mask function
First, since we will be doing this a lot, let's define our own function to calculate a mask given: 1) the original image and 2) a threshold value. We'll give the function an informative name, like get_mask()

In [ ]:
def get_mask(im, threshold):
    # your code here
    return(mask)

Now let's look at the masks generated using these automatic thresholds. 

***
# Part 3: Filtering and background subtraction

To illustrate the need for preprocessing, let's try to automatically threshold the other fluorescent channel of our data

Our simple automatic thresholding fails for the other channel. There are at least two reasons for these failures:  
1) Salt & pepper, speckly noise  
2) A background due to uneven illumination or other optical aberrations

We'll try to get rid of the salt and pepper noise first. We'll do that by __filtering__

We need to import some packages and filters:

### Part 3b) Filtering out noise


Now, we'll try using a median filter to remove the salt-pepper noise. Can you see differences in the images? Play with the filter size and see how that affects the filtering.

### Part 3c) Background subtraction

We still need to remove the un-even illumination. To do this we will subtract a background image, leaving intact the image structure we care about. First we need to compute the background using a MIN() filter.

#### <font color='red'> Exercise 3</font>: How does the calculated background depend on the filter size? What happens if the filter is too small? Too large?

Next, let's substract out the background from the image and visualize the result.

With our pre-processed dataset, let's try the thresholding again.

### Bonus: The limits of global intensity thresholding

We haven't yet tried any processing on our brightfield channel. Let's see how the methods we tried above work on these data

In [ ]:
filter_radius = 12
brt_background = min_filter(brt_data, disk(filter_radius))

brt_bgs = brt_data - brt_background
brt_bgs[brt_bgs<0] = 0

thresh = threshold_otsu(brt_bgs)
mask = get_mask(brt_bgs, thresh)

fig, ax = plt.subplots(1, 3, figsize=(18, 6))
ax[0].imshow(brt_data, cmap='gray')
ax[0].set_title('Image')

ax[1].imshow(brt_bgs, cmap='gray')
ax[1].set_title('Preprocessed')

ax[2].imshow(mask, cmap='gray')
ax[2].set_title('Mask')

In [ ]:
plt.hist(brt_data.flatten(), bins=50);
plt.yscale('log')

#### Edge / contrast-enhancing filters   
The above methods rely on the intensity of each pixel, but we see the cells in the DIC channel with our eyes because of the contrast in the image. To pick out high contrast regions we can try a edge-detection filter

In [ ]:
from skimage.filters import roberts

brt_edge = roberts(brt_data)
thresh = threshold_otsu(brt_edge)
mask = get_mask(brt_edge, thresh)

fig, ax = plt.subplots(1, 3, figsize=(18, 6))
ax[0].imshow(brt_data, cmap='gray')
ax[0].set_title('Image')

ax[1].imshow(brt_edge, cmap='gray')
ax[1].set_title('Roberts filtered')

ax[2].imshow(mask, cmap='gray')
ax[2].set_title('Mask')

In [ ]:
plt.hist(brt_edge.flatten(), bins=50)
plt.axvline(thresh, c='r', label='Otsu threshold') 
plt.yscale('log')